In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
dfsales = pd.read_csv("/home/jian/BiglotsCode/outputs/combinedsales2017-11-11.csv",sep = '|',dtype = 'str')

In [3]:
dftraffics = pd.read_csv("/home/jian/BiglotsCode/outputs/combinedtraffic2017-11-11.csv",
               sep = '|',dtype = 'str')

In [4]:
dfsales['week_end_dt']=pd.to_datetime(dfsales['week_end_dt']).dt.date
dftraffics['week_end_dt']=pd.to_datetime(dftraffics['week_end_dt']).dt.date

In [5]:
dfsales=dfsales[(dfsales['week_end_dt']<=dt.date(2017,11,11)) & (dfsales['week_end_dt']>=(dt.date(2017,11,11)-dt.timedelta(days=(51*7))))]
dftraffics=dftraffics[(dftraffics['week_end_dt']<=dt.date(2017,11,11)) & (dftraffics['week_end_dt']>=(dt.date(2017,11,11)-dt.timedelta(days=(51*7))))]

In [6]:
dfsales.columns

Index(['location_id', 'week_end_dt', 'fiscal_week_nbr', 'gross_sales_amt',
       'gross_transaction_cnt', 'class_code_id', 'class_gross_sales_amt'],
      dtype='object')

In [7]:
len(dfsales['week_end_dt'].unique())

52

In [9]:
dftraffics = dftraffics.fillna(0)

In [10]:
dftraffics['traffic_week'] = 0 
for i in ['traffic_day_1','traffic_day_2', 'traffic_day_3', 'traffic_day_4',
          'traffic_day_5', 'traffic_day_6', 'traffic_day_7']:
    dftraffics[i] = dftraffics[i].astype('float')
    dftraffics['traffic_week'] = dftraffics['traffic_week'] +dftraffics[i]

In [11]:
dfsales = dfsales[dfsales['class_gross_sales_amt']!='?']
dfsales = dfsales [~ dfsales['location_id'].isin(['145','6990'])]
dftraffics = dftraffics [~ dftraffics['location_id'].isin(['145','6990'])]


In [12]:
dfsales=dfsales.drop_duplicates()
dfsales ['class_gross_sales_amt']= dfsales['class_gross_sales_amt'].astype('float')
dfsales ['gross_transaction_cnt']= dfsales['gross_transaction_cnt'].astype('float')

In [13]:
dfsales_sales = dfsales[['location_id','week_end_dt','class_gross_sales_amt']].groupby(['location_id','week_end_dt']).sum()
dfsales_sales.reset_index(inplace=True)

In [14]:
# Remove stores with recorded sales less than 40 weeks of sales data
dfsales_sales_table = dfsales_sales.groupby(['location_id'])['week_end_dt'].count()
dfsales_sales_table=pd.DataFrame(dfsales_sales_table)
dfsales_sales_table.reset_index(inplace=True)

Store_sales_less_40 = dfsales_sales_table[dfsales_sales_table['week_end_dt']<=40]
Store_sales_less_40.reset_index(inplace=True)
print("Stores with sales records less or equal than 40 weeks:",Store_sales_less_40['location_id'].unique())

dfsales_sales = dfsales_sales[dfsales_sales['location_id'].isin(dfsales_sales_table[dfsales_sales_table['week_end_dt']>40]['location_id'])]

Stores with sales records less or equal than 40 weeks: ['1148' '1182' '1913' '1967' '280' '310' '388' '4099' '4113' '4124' '4165'
 '4193' '4280' '4362' '4667' '4669' '4671' '4672' '4673' '507' '5133'
 '5340' '5343' '5344' '5345' '5346' '5347' '5348' '5349' '5350' '5351'
 '5352' '5353' '5354' '5355' '5356' '5357' '5358' '5359' '5360' '824']


In [15]:
dfsales_group = dfsales_sales[['location_id','class_gross_sales_amt']].groupby(['location_id']).sum()
dfsales_group.reset_index(inplace=True)
# dfsales_group['sales_group']=np.where(dfsales_group['class_gross_sales_amt']<=dfsales_group['class_gross_sales_amt'].quantile(0.2),"L",
#                                      np.where(dfsales_group['class_gross_sales_amt']>dfsales_group['class_gross_sales_amt'].quantile(0.8),"H","M"))


In [16]:
# Remove stores with recorded transactions less than 40 weeks of sales data

dfsales_trans = dfsales[['location_id','week_end_dt','gross_transaction_cnt']].drop_duplicates()
dfsales_trans_table = dfsales_trans.groupby(['location_id'])['week_end_dt'].count()
dfsales_trans_table=pd.DataFrame(dfsales_trans_table)
dfsales_trans_table.reset_index(inplace=True)

Store_trans_less_40 = dfsales_trans_table[dfsales_trans_table['week_end_dt']<=40]
Store_trans_less_40.reset_index(inplace=True)
print("Stores with trans records less or equal than 40 weeks:",Store_trans_less_40['location_id'].unique())
dfsales_trans = dfsales_trans[dfsales_trans['location_id'].isin(dfsales_trans_table[dfsales_trans_table['week_end_dt']>40]['location_id'])]

Stores with trans records less or equal than 40 weeks: ['1148' '1182' '1913' '1967' '280' '310' '388' '4099' '4113' '4124' '4165'
 '4193' '4280' '4362' '4667' '4669' '4671' '4672' '4673' '507' '5133'
 '5340' '5343' '5344' '5345' '5346' '5347' '5348' '5349' '5350' '5351'
 '5352' '5353' '5354' '5355' '5356' '5357' '5358' '5359' '5360' '824']


In [17]:
dftraffics = dftraffics[dftraffics['traffic_week']>5]
recent_52_weeks = pd.date_range(start=dt.date(2017,11,11)-dt.timedelta(days=(51*7)), end=dt.date(2017,11,11), freq=dt.timedelta(days=7)) 
lack_weeks_traffic = recent_52_weeks[~recent_52_weeks.isin(dftraffics['week_end_dt'].unique())]
print("Lack weeks for traffic records",lack_weeks_traffic)

Lack weeks for traffic records DatetimeIndex(['2016-12-31', '2017-05-06'], dtype='datetime64[ns]', freq=None)


In [18]:
# Remove stores with recorded traffics less than 5 weeks of traffics data
dftraffics_table = dftraffics[['location_id','week_end_dt']].groupby(['location_id']).count()
dftraffics_table.reset_index(inplace=True)
Store_traffics_less_5 = dftraffics_table[dftraffics_table['week_end_dt']<=5]
Store_traffics_less_5.reset_index(inplace=True)
print("Stores with traffics records less or equal than 5 weeks:",Store_traffics_less_5['location_id'].unique())
dftraffics = dftraffics[dftraffics['location_id'].isin(dftraffics_table[dftraffics_table['week_end_dt']>5]['location_id'])]
dftraffics=dftraffics[['location_id','week_end_dt','traffic_week']]

Stores with traffics records less or equal than 5 weeks: ['4667' '5345' '5352' '5355']


In [28]:
df_ratio = pd.merge(dfsales_trans,dftraffics,how='left',on=['location_id','week_end_dt'])
df_ratio = df_ratio[(~pd.isnull(df_ratio['gross_transaction_cnt']))& (~pd.isnull(df_ratio['traffic_week'])) ]

In [29]:
df_ratio_group = df_ratio[['location_id','gross_transaction_cnt','traffic_week']].groupby(['location_id']).sum()
df_ratio_group.reset_index(inplace=True)
df_ratio_group['ratio']=df_ratio_group['gross_transaction_cnt']/df_ratio_group['traffic_week']
df_ratio_group['ratio_group']=np.where(df_ratio_group['ratio']<=df_ratio_group['ratio'].quantile(0.2),"L",
                                      np.where(df_ratio_group['ratio']>df_ratio_group['ratio'].quantile(0.8),"H","M"))

In [30]:
writer= pd.ExcelWriter('/home/jian/Projects/Big Lots/New Group - 52 weeks prior 2017-11-11/BL_Sales_Ratio_Group_till_2017-11-11.xlsx', engine='xlsxwriter')
df_ratio_group.to_excel(writer,"ratio_group",index=False)
dfsales_group.to_excel(writer,"sales_group",index=False)
dfsales_sales_table.to_excel(writer,"sales_week_count",index=False)
df_ratio.to_excel(writer,"ratio_data",index=False)

writer.save()